In [1]:
import sys
sys.path.insert(0, '..')

In [6]:
print(sys.path)

['..', '/tmp', '/home/quanghung20gg/Documents/hung20gg/ielts_scoring/test_infer', '/home/quanghung20gg/anaconda3/lib/python311.zip', '/home/quanghung20gg/anaconda3/lib/python3.11', '/home/quanghung20gg/anaconda3/lib/python3.11/lib-dynload', '', '/home/quanghung20gg/anaconda3/lib/python3.11/site-packages', '/tmp/tmp2zu2ixn5']


In [1]:
# from huggingface_hub import login
# login() # Get your token here

In [8]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from utils.extract import get_score, clean_output, get_essay
from utils.prompt import rescore, indepth_feedback, until_correct, get_instruction_prompt, get_system_prompt

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "mistralai/Mistral-7B-v0.1" # Super loose
# model_id = 'google/gemma-7b-it'
# model_id = "microsoft/Phi-3-mini-4k-instruct"
mode = 'easy'
if 'llama' in model_id:
    mode = 'harsh'
    
chatbot_role = 'system'
change_mode = True
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map="auto",
# )

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.generation_config.pad_token_id = tokenizer.eos_token_id


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 23.67 GiB of which 70.00 MiB is free. Including non-PyTorch memory, this process has 22.87 GiB memory in use. Of the allocated memory 22.56 GiB is allocated by PyTorch, and 65.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [2]:
EXAMPLES = 'cambridge/1x_3_7'

**Cam 17**

- Test 1: 6.5 7
- Test 2: 6.5 6
- Test 3: 6.5 7
- Test 4: 6 6.5

In [3]:
    
with open('prompt/output_suggestion_format.txt', 'r') as f:
    output_suggestion = f.read()
    
with open(f'sample/{EXAMPLES}/question.txt', 'r') as f:
    essay_topic = f.read()

with open(f'sample/{EXAMPLES}/answer.txt', 'r') as f:
    student_response = f.read()


## Get the System prompt

In [4]:
system_prompt = get_system_prompt(False)
instruction_prompt_1 = get_instruction_prompt(essay_topic, student_response)
messages = [
    # {"role": "user", "content": system_prompt},
    # {"role": chatbot_role, "content": "Sure"},
    {"role": chatbot_role, "content": system_prompt},
    {"role": "user", "content": instruction_prompt_1},
]


## First generate result

In [5]:

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
model_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.6,
    "top_p":0.9,
    "do_sample": True,
}

generation_args = {
    "model_args": model_args,
    "role": chatbot_role,
}


output1 = pipe(messages, **generation_args["model_args"])
output_1_text = output1[0]["generated_text"]
messages.append({"role": "assistant", "content": output_1_text})
print(output_1_text)


**General: 6.5**
Overall Explanation: The essay presents a good discussion of the topic, but lacks depth and coherence in some areas. The writer has attempted to address both views, but the supporting evidence and examples are not consistently strong. The writing is clear, but the vocabulary and grammar could be more varied and accurate.

**Task Response: 6**
Explanation: The essay addresses the topic and presents both views, but the discussion is not as comprehensive as it could be. The writer has not fully explored the implications of each view and has not provided sufficient evidence to support their own opinion. The essay lacks a clear thesis statement and the writer has not effectively linked the different paragraphs together.

**Coherence and Cohesion: 6**
Explanation: The essay has a clear introduction and conclusion, but the body paragraphs could be more cohesive. The writer has used some transitional phrases, but the connections between paragraphs are not always smooth. The te

In [8]:
chatbot_role = 'assistant'
generation_args = {
    "model_args": model_args,
    "role": chatbot_role,
}



In [7]:
print(messages[0]['content'])


    You are an English teaching assistant, and you are good at grading essays, and your students need you for their IELTS Academic essay task 2. You will be given the topic and student's response.
    You should grade the essay general score and in 4 metrics in IELTS Writing, which are Task Response, Coherence and Cohesion, Lexical Resource and Grammatical Range and Accuracy. 
    The overall must be the mean value of 4 metric scores and can be a float value between 0 and 9 (round to .5), but each metric score should be an integer between 0 and 9.

    The formula to calculate the general score is:
    ```
    ( ( Task_Response + Coherence_and_Cohesion + Lexical_Resource + Grammatical_Range_and_Accuracy) // 2 ) / 2 = General_score
    ```


    Recall the IELTS Writing band score criteria.

    

    In each metric, you should give a detailed explanation and point out exactly the student mistakes that led to that score.
    Your output format should be like this:
    **General: score*

In [9]:
adjust_output  = until_correct(pipe, messages, **generation_args)
general, tr, cc, lr, gr = get_score(adjust_output)

In [10]:
print(clean_output(adjust_output))

**General: 6.0**
Overall Explanation: The essay presents a good discussion of the topic, but lacks depth and coherence in some areas. The writer has attempted to address both views, but the supporting evidence and examples are not consistently strong. The writing is clear, but the vocabulary and grammar could be more varied and accurate.

**Task Response: 6**
Explanation: The essay addresses the topic and presents both views, but the discussion is not as comprehensive as it could be. The writer has not fully explored the implications of each view and has not provided sufficient evidence to support their own opinion. The essay lacks a clear thesis statement and the writer has not effectively linked the different paragraphs together.

**Coherence and Cohesion: 6**
Explanation: The essay has a clear introduction and conclusion, but the body paragraphs could be more cohesive. The writer has used some transitional phrases, but the connections between paragraphs are not always smooth. The te

In [11]:
messages = [
    # {"role": "user", "content": system_prompt},
    # {"role": chatbot_role, "content": "Sure, I will be a scoring assistant for your IELTS writing task 2 essay."},
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": instruction_prompt_1},
    {"role": generation_args["role"], "content": clean_output(adjust_output)},
]

In [12]:
if change_mode:
    rescore_output = rescore(pipe, messages, mode, **generation_args)
    rescore_output = until_correct(pipe, messages, **generation_args)
    general, tr, cc, lr, gr = get_score(rescore_output)
    print(rescore_output)

I apologize for the mistake. You are correct that the scores I provided do not add up to a general score of 6.5. Here is the revised output:

**General: 7.0**
Overall Explanation: The essay presents a good discussion of the topic, with some strengths in addressing both views and providing examples. The writing is generally clear, coherent, and well-organized.

**Task Response: 7**
Explanation: The essay addresses the topic and presents both views, with some good supporting evidence and examples. The writer has attempted to address the implications of each view, and the discussion is generally well-structured.

**Coherence and Cohesion: 7**
Explanation: The essay has a clear introduction and conclusion, and the body paragraphs are generally well-organized. The writer has used some transitional phrases to link ideas together, and the text flows smoothly.

**Lexical Resource: 7**
Explanation: The essay demonstrates a good range of vocabulary, with some sophisticated and nuanced expression

### Provide in-depth feedback

In [10]:
first_output = clean_output(adjust_output)
if change_mode:
    first_output = clean_output(rescore_output)
messages = [
    {"role": generation_args["role"], "content": system_prompt},
    {"role": "user", "content": instruction_prompt_1},
    {"role": generation_args["role"], "content": first_output},
]
rcm_output = indepth_feedback(pipe, messages, **generation_args)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [11]:
print(rcm_output)

**General: 6.5**
Overall Explanation: The essay provides a good overview of the topic, discussing both sides of the argument and presenting a clear opinion. While the writing is clear, the vocabulary is limited, and there are some errors in grammar and punctuation. The essay could benefit from more depth and analysis, but it is generally well-structured and easy to follow.

**Task Response: 7**
Explanation: The essay addresses the topic adequately, presenting both sides of the argument and the writer's own opinion. However, the discussion is somewhat superficial, and the writer could benefit from providing more supporting details and examples to strengthen their argument. For instance, the writer mentions that advertisements can be successful in persuading people to buy things, but they do not provide any specific examples or statistics to support this claim. Additionally, the writer's own opinion is not fully developed, and they could benefit from providing more evidence and analysis 

## Making adjustment to the essay

In [12]:
MAX_TARGET = 7.5
ADJUSTMENT_MODE = 'close_to_original'

In [13]:
if ADJUSTMENT_MODE =='close_to_original' and change_mode:
    general, tr, cc, lr, gr = get_score(adjust_output)
    first_output = clean_output(adjust_output)
    messages = [
        {"role": generation_args["role"], "content": system_prompt},
        {"role": "user", "content": instruction_prompt_1},
        {"role": generation_args["role"], "content": first_output},
    ]
    indepth_feedback(pipe, messages, **generation_args)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [14]:
CURRENT_TARGET = general

if CURRENT_TARGET <= MAX_TARGET - 0.5:
    messages.append({"role": "user", "content": f"Making adjustment directly into the essay to improve at least 1 score in each metric. {output_suggestion}"})
    suggest_essay = pipe(messages, **generation_args["model_args"])
    
else: 
    messages.append({"role": "user", "content": f"Making adjustment directly into the essay to optimize the essay. {output_suggestion}"})
    suggest_essay = pipe(messages, **generation_args["model_args"])


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [15]:
print(suggest_essay[0]['generated_text'])

**Suggesting changes:**

**Task Response:**

* Provide more specific examples and evidence to support the claims made in the essay.

**Coherence and Cohesion:**

* Use transitional phrases to connect ideas between paragraphs, such as "However," "In addition," or "Furthermore."
* Use more varied sentence structures to make the writing more engaging.

**Lexical Resource:**

* Use more nuanced and varied vocabulary to describe the effects of advertisements, such as "subconscious influence" or "mental persuasion."

**Grammatical Range and Accuracy:**

* Use more complex sentence structures, such as compound or complex sentences, to add variety to the writing.
* Check for errors in grammar and punctuation, such as missing articles or incorrect verb tenses.

**Revised Essay:**

Advertisement has always been a crucial part of the world of marketing. Throughout the decades, we have seen a significant increase in the amount of advertisements, whether it is on the media like television or widesp

In [16]:
print(get_essay(suggest_essay[0]['generated_text']))



Advertisement has always been a crucial part of the world of marketing. Throughout the decades, we have seen a significant increase in the amount of advertisements, whether it is on the media like television or widespread through social network platforms. The goal of advertisements is to get consumers to buy a targeted product, and while this method has been proven considerably successful generally, some people view it as too prevalent to catch the consumers' attention any more.

Advertisements can act as a strong persuasion device to seemingly hypnotize people into buying goods and services. This is so because of the tactics placed in the messages, such as showing people having a good time together when using a particular product, using bandwagon, showing only the upsides of usage, and applying compare and contrast strategies to show the effects of using the product and make it stand out. Even if people do not know it, these messages are repeated several times and soon it may brainw